In [1]:
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from tensorflow.keras.layers import LSTM

import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

import h5py

c:\Users\Wildan Mufid R\miniconda3\envs\main\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
c:\Users\Wildan Mufid R\miniconda3\envs\main\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.11.0 and strictly below 2.14.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.14.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not fi

In [48]:
target = 'bilstm'
LR = []
BZ = []
COM = []
Ep = []
train_acc = []
val_acc = []
train_loss = []
val_loss = []
avg_acc = []
avg_loss = []

for mod in [1,2,3]:
    for bz in [16,32,64]:
        for red in [5]:
            for lr in [1E-03, 1E-04, 1E-05]:
                with h5py.File(f'logs{mod}/{bz}/history_{target}_dset{red}_{lr}.h5','r') as hf:
                    LR.append(lr)
                    BZ.append(bz)
                    COM.append(mod)
                    Ep.append(len(np.array(hf.get('accuracy'))))
                    train_acc.append(np.array(hf.get('accuracy'))[-1])
                    val_acc.append(np.array(hf.get('val_accuracy'))[-1])
                    train_loss.append(np.array(hf.get('loss'))[-1])
                    val_loss.append(np.array(hf.get('val_loss'))[-1])
                    avg_acc.append((np.array(hf.get('accuracy'))[-1] + np.array(hf.get('val_accuracy'))[-1]) / 2)
                    avg_loss.append((np.array(hf.get('loss'))[-1] + np.array(hf.get('val_loss'))[-1]) / 2)

In [49]:
d = {'Lr': LR, 'Batch Size': BZ, 'Combination': COM, 'Epoch': Ep, 'Train Acc': train_acc, 'Val Acc': val_acc, 'Train Loss': train_loss, 'Val Loss': val_loss, 'Avg Acc': avg_acc, 'Avg Loss': avg_loss}
df = pd.DataFrame(data=d)

In [50]:
df = df.sort_values(by=['Avg Acc', 'Avg Loss'], ascending=[False,True])
df

,Lr,Batch Size,Combination,Epoch,Train Acc,Val Acc,Train Loss,Val Loss,Avg Acc,Avg Loss
21,0.00100,32,3,66,1.000000,0.980488,0.002443,0.082756,0.990244,0.042600
12,0.00100,32,2,30,1.000000,0.980488,0.002993,0.129185,0.990244,0.066089
10,0.00010,16,2,24,0.995098,0.985366,0.036366,0.100582,0.990232,0.068474
19,0.00010,16,3,47,0.998775,0.975610,0.007045,0.129028,0.987192,0.068037
15,0.00100,64,2,51,1.000000,0.970732,0.001363,0.198958,0.985366,0.100160
24,0.00100,64,3,43,0.998775,0.970732,0.003261,0.183788,0.984753,0.093525
9,0.00100,16,2,45,0.998775,0.970732,0.004181,0.243318,0.984753,0.123750
6,0.00100,64,1,48,1.000000,0.965854,0.003884,0.133925,0.982927,0.068904
22,0.00010,32,3,21,0.996324,0.965854,0.018218,0.154482,0.981089,0.086350
13,0.00010,32,2,30,0.996324,0.965854,0.066472,0.182702,0.981089,0.124587


In [51]:
df.to_csv('logs_bilstm5.csv')

In [92]:
with h5py.File('logs2/64/history_bilstm_dset2_0.0001.h5','r') as hf:
    print(hf.keys())
    accuracy = np.array(hf.get('accuracy'))
    f1_score = np.array(hf.get('f1_score'))
    loss = np.array(hf.get('loss'))
    val_accuracy = np.array(hf.get('val_accuracy'))
    val_f1_score = np.array(hf.get('val_f1_score'))
    val_loss = np.array(hf.get('val_loss'))

<KeysViewHDF5 ['accuracy', 'f1_score', 'loss', 'val_accuracy', 'val_f1_score', 'val_loss']>


In [93]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(1,len(accuracy)+1), y=accuracy,
                    mode='lines',
                    name='Akurasi'))
fig.add_trace(go.Scatter(x=np.arange(1,len(val_accuracy)+1), y=val_accuracy, 
                    mode='lines',
                    name='Akurasi Validasi'))
# fig.add_trace(go.Scatter(x=np.arange(1,len(f1_score)+1), y=np.average(f1_score, axis=1),
#                     mode='lines',
#                     name='F1-Score'))
# fig.add_trace(go.Scatter(x=np.arange(1,len(val_f1_score)+1), y=np.average(val_f1_score, axis=1),
#                     mode='lines',
#                     name='F1-Score Validasi'))
fig.add_trace(go.Scatter(x=np.arange(1,len(loss)+1), y=loss,
                    mode='lines',
                    name='Loss'))
fig.add_trace(go.Scatter(x=np.arange(1,len(val_loss)+1), y=val_loss, 
                    mode='lines',
                    name='Loss Validasi'))  


fig.update_layout(height=600, width=1000)
fig.update_layout(xaxis_title='Epoch', yaxis_title='Nilai')
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
# fig.add_vline(
#         x=np.argmax(val_accuracy)+1, line_width=1, line_dash="dash", 
#         line_color="red")

# fig.add_annotation(x=np.argmax(val_accuracy)+1, y=np.max(val_accuracy),
#             text=f"Model Akurasi Validasi Tertinggi (Epoch ke-{np.argmax(val_accuracy)+1})",
#             ay=100,
#             ax=150,
#             showarrow=True,
#             arrowhead=1,arrowwidth=2,arrowsize=1,
#             font=dict(
#                 color="black",
#                 size=12
#             ),
#             bgcolor="#FFD54F",borderwidth=1,borderpad=4,)

fig.update_layout(margin=dict(l=10, r=20, t=10, b=10))

fig.show()

In [50]:
loss[-1]

0.0004845588409807533

In [367]:
import plotly.io as pio
pio.write_image(fig, 'asset/bilstm dset2.png',scale=3, width=1080, height=648)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(1,len(loss)+1), y=loss,
                    mode='lines',
                    name='Loss'))
fig.add_trace(go.Scatter(x=np.arange(1,len(val_loss)+1), y=val_loss, 
                    mode='lines',
                    name='Loss Validasi'))

fig.update_layout(height=600, width=1000)
fig.update_layout(xaxis_title='Epoch', yaxis_title='Nilai')
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))

fig.update_layout(margin=dict(l=10, r=20, t=10, b=10))

fig.show()

In [212]:
import plotly.io as pio
pio.write_image(fig, 'asset/lstm dset1 loss.png',scale=2, width=1080, height=648)
# pio.write_image(fig, 'asset/image.png',scale=2, width=1080, height=648)

In [26]:
val_loss[[19,29]]

array([0.12010226, 0.09312012])

In [24]:
np.average(f1_score[[19,29]], axis=1)

array([0.9762738, 0.990039 ], dtype=float32)

In [56]:
with h5py.File('data/.preprocessed_data/datasets1.h5','r') as hf:
    y_train = np.array(hf.get('y_train'))
    y_val = np.array(hf.get('y_val'))

OHE = OneHotEncoder(sparse=False).fit(y_train.reshape(-1, 1))

decoder = {i : sign for i, sign in enumerate(OHE.categories_[0])}

y_train = OHE.transform(y_train.reshape(-1, 1))
y_val = OHE.transform(y_val.reshape(-1, 1))

c:\Users\Wildan Mufid R\miniconda3\envs\main\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [82]:
with h5py.File('data/.preprocessed_data/datasets5.h5','r') as hf:
    x_train = np.array(hf.get('x_train'))
    x_val = np.array(hf.get('x_val'))

In [89]:
with h5py.File('logs/history_bilstm_dset5.h5','r') as hf:
    print(hf.keys())
    accuracy = np.array(hf.get('accuracy'))
    f1_score = np.array(hf.get('f1_score'))
    loss = np.array(hf.get('loss'))
    val_accuracy = np.array(hf.get('val_accuracy'))
    val_loss = np.array(hf.get('val_loss'))

<KeysViewHDF5 ['accuracy', 'f1_score', 'loss', 'relative', 'val_accuracy', 'val_loss']>


In [90]:
print(np.argmax(val_accuracy))
print(accuracy[np.argmax(val_accuracy)])
print(val_accuracy[np.argmax(val_accuracy)])
print(np.average(f1_score[np.argmax(val_accuracy)]))
print(loss[np.argmax(val_accuracy)])
print(val_loss[np.argmax(val_accuracy)])

22
0.9865196347236633
0.9853658676147461
0.98620385
0.04573823884129524
0.04791378229856491


In [91]:
model3 = tf.keras.models.load_model(f'models/best_bilstm_dset5.h5', custom_objects={'Addons>F1Score': tfa.metrics.F1Score(9)})

In [92]:
history = model3.evaluate(x_val, y_val)

7/7 [==============================] - 4s 261ms/step - loss: 0.0479 - accuracy: 0.9854 - f1_score: 0.9855
